In [1]:
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model

%matplotlib inline

Using TensorFlow backend.


### Functions for preprocessing Images

In [2]:
# path for the HaarCascade
cascade_path = "./haarcascade_frontalface_alt2.xml"

In [9]:
# preprocessing
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def load_and_align_images(filepaths, margin):
    cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    aligned_images = []
    labels = []
    for filepath in filepaths:
        img = imread(filepath)

        faces = cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3)
        if len(faces) == 0:
            continue
            
        (x, y, w, h) = faces[0]
        cropped = img[y-margin//2:y+h+margin//2, x-margin//2:x+w+margin//2, :]
        
        if (cropped.shape[0] == 0):
            continue
        
        aligned = resize(cropped, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
        
        # need to manunally change it - change it - use a parameter
        # for test dataset
        # labels.append(filepath.split("/")[-2])
        
#         for train dataset
        labels.append(filepath.split("/")[-1])
    
    return np.array(aligned_images), labels

### Functions for calculating word embeddings i.e. 128 dimensional vector

In [4]:
# functions for calculating word embeddings i.e. 128 dimensional vector
def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

def calc_embs(filepaths, margin=10, batch_size=1):
    aligned_images, labels = load_and_align_images(filepaths, margin)
    aligned_images = prewhiten(aligned_images)
    pd = []
    for start in range(0, len(aligned_images), batch_size):
        pd.append(model.predict_on_batch(aligned_images[start:start+batch_size]))
    embs = l2_normalize(np.concatenate(pd))

    return embs, labels

### Loading the test dataset

In [5]:
# reading classes names
image_dir_basepath_test = "./test/"
names = os.listdir(image_dir_basepath_test)
image_size = 160

In [6]:
# pretrained model path
model_path = "./keras-facenet-20190502T135832Z-001/keras-facenet/model/facenet_keras.h5"
model = load_model(model_path)

/home/vasu/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [8]:
# getting encoding for test images
data = {}

for name in names:
    image_dirpath = image_dir_basepath_test + name
    image_filepaths = [os.path.join(image_dirpath, f) for f in os.listdir(image_dirpath)]
    embs, labels = calc_embs(image_filepaths)
    
    for i in range(len(embs)):
        data['{}{}'.format(name, i)] = {'image_filepath' : labels[i], 'emb' : embs[i]}

/home/vasu/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


### Getting encodings for training dataset

In [10]:
# getting the encodings for training data
train_data = {}
image_dirpath = "./training"
image_filepaths = [os.path.join(image_dirpath, f) for f in os.listdir(image_dirpath)]

embs, labels = calc_embs(image_filepaths)
    
for i in range(len(embs)):
    train_data['{}{}'.format(name, i)] = {'image_filepath' : labels[i], 'emb' : embs[i]}

## NEAREST NEIGHBOUR CALCULATION

In [12]:
# now for every test data we find the closest image to it
count = 0
for k, v in data.items():
    min_dist = 1000
    identity = None
    for k_train, v_train in train_data.items():
        dist = np.linalg.norm(v['emb'] - v_train['emb'])
        if dist < min_dist:
            min_dist = dist
            identity = v_train['image_filepath']
    
    if (v['image_filepath'].lower() in identity.lower()):
        count = count + 1
    else:
        pass
        # print (v['image_filepath'] + "   " + identity)

### Accuracy

In [13]:
count / len(data)

0.9192546583850931